In [1]:
import os
os.chdir('/home/fraza0/Desktop/MEI/TESE/nonverbal_communication_analysis')
print("Working Directory:", os.getcwd())
import ipywidgets as widgets

Working Directory: /home/fraza0/Desktop/MEI/TESE/nonverbal_communication_analysis


In [2]:
import pandas as pd
from pandas.io.json import json_normalize
import json
from pprint import pprint
import threading
from IPython.display import display
import ipywidgets as widgets
import time
from nonverbal_communication_analysis.environment import OPENFACE_OUTPUT_DIR, OPENPOSE_OUTPUT_DIR, VALID_OUTPUT_FILE_TYPES
from nonverbal_communication_analysis.utils import fetch_files_from_directory, filter_files

In [3]:
per_frame_metrics = {
    'n_people': 0,
    'valid_face_keypoints': 0,
    'total_valid_face_keypoints': 0,
    'valid_pose_keypoints': 0,
    'total_valid_pose_keypoints': 0,
}

# [{frame, per_frame_metrics}]
vid_metrics = []

In [4]:
groups_list = os.listdir(OPENPOSE_OUTPUT_DIR)

GROUP_ID = widgets.Dropdown(
    options=groups_list,
    value=groups_list[-2],
    description='Group:',
    disabled=False,
)
display(GROUP_ID)

Dropdown(description='Group:', index=3, options=('3CLC9VWR_output_OLD.tar.gz', '1403201803_output', 'walking_h…

In [7]:
_dir = OPENPOSE_OUTPUT_DIR+GROUP_ID.value+"/output_pc1/"
input_files = [_dir+file for file in filter_files(fetch_files_from_directory([_dir]), valid_types = VALID_OUTPUT_FILE_TYPES)]
total_files = len(input_files)
print("Found: %s files" % total_files)

Found: 72178 files


In [8]:
CONFIDENCE_THRESHOLD = widgets.FloatSlider(
    value=0.50,
    min=0,
    max=1.0,
    step=0.05,
    description='Minimum Keypoint Confidence:',
    disabled=False,
    orientation='horizontal',
    readout=True,
    readout_format='.1f'
)
display(CONFIDENCE_THRESHOLD)

FloatSlider(value=0.5, description='Minimum Keypoint Confidence:', max=1.0, readout_format='.1f', step=0.05)

In [9]:
def calculate_person_valid_keypoints(person_df):
    valid_person_pose_keypoints = 0
    person_entry = [person_df[x:x+3] for x in range(0, len(person_df), 3)]
    for pose_entry in person_entry:
        confidence = pose_entry[2]
        if confidence > CONFIDENCE_THRESHOLD.value:
            valid_person_pose_keypoints += 1
    return valid_person_pose_keypoints

In [27]:
%%time

progress = widgets.FloatProgress(value=0.0, min=0.0, max=total_files)
def get_metrics(progress, input_files):
    counter = 0
    for file in input_files[:]:
        counter += 1
        progress.value = float(counter/total_files)
        with open(file) as json_data:
            data = json.load(json_data)
            df = json_normalize(data['people'])
            per_frame_metrics['n_people'] = len(df['person_id'])
            per_frame_metrics['valid_face_keypoints'] = dict(df['face_keypoints_2d'].apply(calculate_person_valid_keypoints))
            per_frame_metrics['total_valid_face_keypoints'] = sum(per_frame_metrics['valid_face_keypoints'].values())
            per_frame_metrics['valid_pose_keypoints'] = dict(df['pose_keypoints_2d'].apply(calculate_person_valid_keypoints))
            per_frame_metrics['total_valid_pose_keypoints'] = sum(per_frame_metrics['valid_pose_keypoints'].values())
        vid_metrics.append({counter: per_frame_metrics})
        
thread = threading.Thread(target=get_metrics, args=(progress,input_files))
display(progress)
thread.start()

FloatProgress(value=0.0, max=72178.0)

CPU times: user 42.3 ms, sys: 12.3 ms, total: 54.6 ms
Wall time: 39 ms
9901
9902
9903
9904
9905
9906
9907
9908
9909
9910
9911
9912
9913
9914
9915
9916
9917
9918
9919
9920
9921
9922
9923
9924
9925
9926
9927
9928
9929
9930
9931
9932
9933
9934
9935
9936
9937
9938
9939
9940
9941
9942
9943
9944
9945
9946
9947
9948
9949
9950
9951
9952
9953
9954
9955
9956
9957
9958
9959
9960
9961
9962
9963
9964
9965
9966
9967

In [265]:
# num_people_time = [n.value for n in vid_metrics]